In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer
from google import genai
import json
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [2]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [6]:
client = genai.Client()

In [7]:
def cosine_similarity(a, b):
    dot_prod = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)

    if norm_a == 0 or norm_b == 0:
        return 0

    return dot_prod / (norm_a * norm_b)

In [8]:
def retrieve(query, ingreidents, k=3):
    query_emb = model.encode(query)

    scores = []
    for ing in ingreidents:
        score = cosine_similarity(query_emb, np.array(ing["embedding"]))

        scores.append((score, ing))

    scores.sort(reverse=True, key=lambda x: x[0])
    return [ing for _, ing in scores[:k]]

In [9]:
def build_context(retrieved):
    lines = []
    for r in retrieved:
        lines.append(
            f"{r['name']} → {r['evidence_strength']} evidence; "
            f"{', '.join(r['health_concern_type']).lower()}"
        )
    return "\n".join(lines)


In [ ]:
# test

with open("embeddings.json", "r") as f:
    ingredients = json.load(f)

user_query = "palm oil, added sugar, emulsifier"

retrieved = retrieve(user_query, ingredients, k=3)
context = build_context(retrieved)

prompt = f"""
You are an expert AI-health co-pilot helping consumers make sense of the ingredients
in food products they want to buy. Your aim is to provide quick insights about the product to the consumers.

Rules:
- Do not main absolute claims about diseases.
- Do not just repeat the ingredients list.
- Do use simple english words so that a consumer easily understands.
- Do explain tradeoffs, not just the risks.
- Do take uncertainty into consideration.
- Answer only based on the given context.
- If context is little or none, say so.
- Keep the response short and simple.


Ingredients insights:
{context}
"""

response = client.models.generate_content(model="gemini-2.5-flash", contents=prompt)

print(response.text)

This product contains palm oil, which adds fat but has moderate concerns regarding its fat quality. It also includes emulsifiers (E471, E472) that help with texture, though there's mixed and emerging evidence suggesting a potential impact on gut health. These ingredients are commonly found in ultra-processed foods, which is something to keep in mind.
